<a href="https://colab.research.google.com/github/wlrma0108/HuggingFace/blob/main/Bert_incomplete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers seqeval[gpu]
!pip3 install torch

     |████████████████████████████████| 3.8 MB 19.0 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 596 kB 53.9 MB/s 
     |████████████████████████████████| 67 kB 7.3 MB/s 
     |████████████████████████████████| 895 kB 65.2 MB/s 
     |████████████████████████████████| 6.5 MB 59.4 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=7b90e351de6b3bfd3369c735cf0fbf1b937cc3b7bf0b3f2881e969776b163829
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer,BertConfig,BertForTokenClassification
from torch import cuda

In [3]:
device ='cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [6]:
data=pd.read_csv("ner_datasetreference.csv", encoding='unicode_escape')
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [7]:
print("Number of tags: {}".format(len(data.Tag.unique())))
frequencies= data.Tag.value_counts()
frequencies

Number of tags: 17


O        184276
B-geo      7695
B-tim      4099
B-org      4050
I-per      3734
B-gpe      3597
B-per      3568
I-org      3274
I-geo      1574
I-tim      1236
B-art       111
B-eve        89
I-eve        71
I-gpe        68
I-art        65
B-nat        53
I-nat        22
Name: Tag, dtype: int64

In [8]:
tags={}
for tag, count in zip(frequencies.index,frequencies):
  if tag !="0":
    if tag[2:5] not in tags.keys():
      tags[tag[2:5]]=count
    else:
      tags[tag[2:5]] +=count
  continue

print(sorted(tags.items(),key=lambda x:x[1],reverse=True))

[('', 184276), ('geo', 9269), ('org', 7324), ('per', 7302), ('tim', 5335), ('gpe', 3665), ('art', 176), ('eve', 160), ('nat', 75)]


In [9]:
entities_to_remove=["B-art","I-art","B-eve","I-eve","B-nat","I-nat"]
data=data[~data.Tag.isin(entities_to_remove)]
data.head()
data=data.fillna(method='ffill')

In [10]:
data.head()


,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [11]:
data['sentence']=data[['Sentence #','Word','Tag']].groupby(['Sentence #'])['Word'].transform(lambda x : ' '. join(x))
data['word_labels']=data[['Sentence #', 'Word','Tag']].groupby(['Sentence #'])['Tag'].transform(lambda x : ','.join(x))
data.head()

,Sentence #,Word,POS,Tag,sentence,word_labels
0,Sentence: 1,Thousands,NNS,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-..."
1,Sentence: 1,of,IN,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-..."
2,Sentence: 1,demonstrators,NNS,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-..."
3,Sentence: 1,have,VBP,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-..."
4,Sentence: 1,marched,VBN,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-..."


In [14]:
labels_to_ids ={k:v for v, k in enumerate(data.Tag.unique())}
ids_to_labels={v:k for v,k in enumerate(data.Tag.unique())}
labels_to_ids


{'B-geo': 1,
 'B-gpe': 2,
 'B-org': 5,
 'B-per': 3,
 'B-tim': 7,
 'I-geo': 4,
 'I-gpe': 9,
 'I-org': 6,
 'I-per': 8,
 'I-tim': 10,
 'O': 0}

In [19]:
data=data[["sentence","word_labels"]].drop_duplicates().reset_index(drop=True)
data.head()

,sentence,word_labels
0,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-..."
1,Families of soldiers killed in the conflict jo...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-per,O,O,..."
2,They marched from the Houses of Parliament to ...,"O,O,O,O,O,O,O,O,O,O,O,B-geo,I-geo,O"
3,"Police put the number of marchers at 10,000 wh...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
4,The protest comes on the eve of the annual con...,"O,O,O,O,O,O,O,O,O,O,O,B-geo,O,O,B-org,I-org,O,..."


In [20]:
len(data)

9931

In [22]:
data.iloc[13].sentence

"Two Germans and four Nigerian oil workers were kidnapped by armed militants during a raid on a boat in Nigeria 's southern oil-rich Delta region ."

In [23]:
data.iloc[13].word_labels

'O,B-gpe,O,O,B-gpe,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-geo,O,O,O,B-geo,O,O'

In [24]:
MAX_LEN=128
TRAIN_BATCH_SIZE=4
VALID_BATCH_SIZE=2
EPOCHS=1
LEARNING_RATE=1e-05
MAX_GRAD_NORM=10
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [25]:
def tokenize_and_preserve_labels(sentence, text_labels,tokenizer):
  tokenized_sentence=[]
  labels=[]
  sentence=sentence.strip()
  for word, label in zip(sentence.split(), text_labels.split(",")):
    tokenized_word=tokenizer.tockenize(word)
    n_subwords = len(tokenized_word)
    tokenized_sentence.extend(tokenized_word)
    labels.extend([label]*n_subwords)

  return tokenized_sentence, labels